## Predicting Zillow
By: Scott Schmidl, Data Scientist
12/16/2021

### Goal
The goal of this project is to understand what affects the price of a house on Zillow and predict, within a range, the price of a house.

### Description
I want to predict housing prices as accuractely as possible to help sellers and buyers maximize their quality of life.

### Initial Questions
1) 
2) 
3) 
4) 

### Data Dictionary
<table>
<thead><tr>
<th>Target</th>
<th>Meaning</th>
</tr>
</thead>
<tbody>
<tr>
<td>home_tax_value</td>
<td>Home price</td>
</tr>
</tbody>
</table>

<table>
<thead><tr>
<th>Variable</th>
<th>Meaning</th>
</tr>
</thead>
<tbody>
<tr>
<td>bedrooms</td>
<td>How many bedrooms are in a house</td>
</tr>
<tr>
<td>bathrooms</td>
<td>How many bathrooms are in a house</td>
</tr>
<tr>
<td>square_feet</td>
<td>Size of the house</td>
</tr>
<tr>
<td>year_built</td>
<td>Year in which the house was built</td>
</tr>
<tr>
<td>fips</td>
<td>County Code</td>
</tr>
<tr>
<td>county</td>
<td>County in which the house is located</td>
</tr>
</tbody>
</table>

### Wrangling Zillow
- To wrangle the Zillow data, I used the Zillow database in our MySQL server, and selected `bedroomcnt`, `bathroomcnt`, `calculatedfinishedsquarefeet`, `taxvaluedollarcnt`, `yearbuilt`, `taxamount`, `fips` for single famiy residences
- Please see wrangle.py for full query details
- Columns were renamed for readability
- The percentage of rows that had na's was small enough that I felt comfortable dropping the rows
- Houses that had zero bathrooms were dropped
- Houses with a size under 200 square feet were dropped
- I handled outliers by removeing a property outside of three standard deviations
- The `taxamount` column was dropped due to it being a potential for data leakage
- A `county` column was created from the `fips` codes

In [1]:
# import Wrangle module
from wrangle import Wrangle

# Instantiate Wrangle class and pull in data from csv or MySQL
train, validate, test = Wrangle().wrangle_zillow()

### Preparing Zillow
To get my X and y sets:
- Dropped the following columns: `home_tax_value`, `fips`, `year_built`, `county`
- Scaled the data by fitting on the X training set only and transforming all three X datasets

In [3]:
# import prepare and pandas module
from prepare import Prepare
import pandas as pd

# instaniate Prepare class and prep data
# This step preps converts train, val, and test into X and y machine learning.
p = Prepare()
(X_train, y_train), (X_val, y_val), (X_test, y_test) = p.get_Xy(train, validate, test)

# This step scales the data for easier use by the algorithm
X_train_scaled, X_val_scaled, X_test_scaled, _ = p.scaling(X_train, X_val, X_test)